In [1]:
! cd ../../kamiwaza-sdk && pip install -e .

Obtaining file:///Users/tylerhouchin/Desktop/kamiwaza-core/kamiwaza/kamiwaza-sdk
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for kamiwaza (pyproject.toml) ... done
  Created wheel for kamiwaza: filename=kamiwaza-0.1.4-0.editable-py3-none-any.whl size=8245 sha256=d81bdd6ed48558166eb9f32dbd7c2edac9e14a96b4396ffd0cb422bb6a75b8a1
  Stored in directory: /private/var/folders/yj/9j0gvmw94m78t21q_rkctb340000gn/T/pip-ephem-wheel-cache-99_vssqs/wheels/8a/dd/3a/d8210e83121acf296d2452151eb40b5706e8c740f230adc3ca
Successfully built kamiwaza
  Attempting uninstall: kamiwaza
    Found existing installation: kamiwaza 0.1.4
    Not uninstalling kamiwaza at /Users/tylerhouchin/Desktop/kamiwaza-core/kamiwaza/kamiwaza-sdk, outside environment /Users/tylerhouchin/Desktop/kamiwaza-core/kamiwaza/notebook-venv
    Can't uninstal

# Set up Kamiwaza Client SDK

In [2]:
from kamiwaza_client import KamiwazaClient
from kamiwaza_client.schemas.ingestion import IngestionConfig
from kamiwaza_client.schemas.retrieval import RetrieveRelevantChunksRequest
from kamiwaza_client.services.retrieval import RetrievalService

import os

# Initialize the client
client = KamiwazaClient("http://localhost:7777/api/")

# Ingest our data. 
This will:
1. Load data into catalog
2. Chunk with offset tracking
3. Embed all chunks
4. Load into vectordb

In [3]:
!ls './pet_store_docs'

break-policy.md                popular-brands-overview.md
company-overview.md            return-policy-for-pet-food.md
customer-service-guidelines.md sanitation-protocols.md
employee-discount-policy.md    store-safety-protocols.md
furry_friends.webp             time-off-requests.md
handling-live-animals.md


In [4]:
dataset_path = './pet_store_docs'
collection_name = "pet_store_docs_demo"

ingestion_config = IngestionConfig(
    dataset_path=dataset_path,
    description="pet_store_docs_demo",
    collection_name=collection_name,
    platform='file'
)

client.ingestion.ingest(ingestion_config)

Processing all 10 files found
Processing file 1/10: handling-live-animals.md


2025-01-28 16:18:45,276 - kamiwaza_client.services.embedding - INFO - Starting embedding generation for 534 chunks (batch size: 32)


Processing file 2/10: store-safety-protocols.md
Processing file 3/10: popular-brands-overview.md
Processing file 4/10: company-overview.md
Processing file 5/10: sanitation-protocols.md
Processing file 6/10: employee-discount-policy.md
Processing file 7/10: time-off-requests.md
Processing file 8/10: return-policy-for-pet-food.md
Processing file 9/10: customer-service-guidelines.md
Processing file 10/10: break-policy.md

Finished processing files:
Total files processed: 10
Total chunks created: 534
Total metadata entries: 534
Failed files: 0

Processing vector batch 1/1


2025-01-28 16:19:02,545 - kamiwaza_client.services.embedding - INFO - Successfully generated embeddings for 534 chunks


Processed 534/534 vectors

Completed vector insertion


# Do retrieval for a query

In [5]:
query = "what is the vacation policy?"

retrieval_request = RetrieveRelevantChunksRequest(
    collections=[collection_name],
    query=query,
    max_results=5
)

In [6]:
results = client.retrieval.retrieve_relevant_chunks(retrieval_request)

In [7]:
for r in results:
    print(r)

('chunks', [ChunkData(source='/Users/tylerhouchin/Desktop/kamiwaza-core/kamiwaza/notebooks/rag-demo/pet_store_docs/time-off-requests.md', offset=0, data='**FurryFriends Pet Supplies: Time-Off Request Policy**\n\n---\n\nAt FurryFriends Pet Supplies, we understand the importance of work-life balance and strive to support our employees in managing their personal and professional responsibilities. This Time-Off Request Policy outlines the procedures and guidelines for requesting time off, ensuring that both employee needs and business operations are effectively balanced.\n\n---\n\n### **1. Requesting Time Off in Advance**\n\n**Advance Notice:**\n- **Standard Requests:** Employees are encouraged to submit time-off requests at least **two weeks** in advance to allow adequate time for scheduling and coverage.\n- **Long-Term Planning:** For extended vacations or significant personal events, providing as much notice as possible is appreciated to facilitate planning.\n\n**Submission Process:**\n

# Do RAG ... that was easy

In [8]:
deployments = client.serving.list_deployments()
valid_deployment = None
for deployment in deployments:
    if deployment.status == 'DEPLOYED' and deployment.instances:
        valid_deployment = deployment
        print(f"Found a deployment of {valid_deployment.m_name} at port {valid_deployment.lb_port} - using it")
        break
if valid_deployment is None:
    print("No valid deployments found.")

Found a deployment of Qwen2.5-7B-Instruct-GGUF at port 51101 - using it


In [9]:
import openai
from openai import OpenAI
import httpx

openai_client = OpenAI(
    api_key="local",
    base_url=f"http://localhost:{valid_deployment.lb_port}/v1",
    http_client=httpx.Client()
)

def rag_inference(query: str, collection_name: str, retrieval_service: RetrievalService):
    retrieval_request = RetrieveRelevantChunksRequest(
        collections=[collection_name],
        query=query,
        max_results=8
    )
    try:
        results = retrieval_service.retrieve_relevant_chunks(retrieval_request)
    except Exception as e:
        print(f"Failed to retrieve relevant chunks: {e}")
        return
    
    context_messages = [
        {"role": "system", "content": "You are a helpful assistant. Here are relevant pieces of information. Be concise."}
    ]
    referenced_files = set()
    
    for chunk in results.chunks:
        context_messages.append(
            {"role": "system", "content": f"Source: {chunk.source}, Chunk ID: {chunk.offset}, Data: {chunk.data}"}
        )
        referenced_files.add(chunk.source)
    
    context_messages.append({"role": "user", "content": query})
    
    try:
        chat_completion = openai_client.chat.completions.create(
            model="local-model",
            messages=context_messages,
            max_tokens=150
        )
        response_content = chat_completion.choices[0].message.content
    except Exception as e:
        print(f"An error occurred during inference: {e}")
        return
    
    print("AI Response:")
    print(response_content)
    print("\nReferenced Files:")
    for file in referenced_files:
        print(f"- {file}")

In [13]:
def default_inference(query: str):
    """
    Make a direct inference call to the LLM without RAG context.
    
    Args:
        query (str): The user's query to send to the model
    """
    try:
        chat_completion = openai_client.chat.completions.create(
            model="local-model",
            messages=[
                {"role": "system", "content": "You are a helpful assistant. Be concise. Do not hallucinate. If you do not know the answer, tell the user."},
                {"role": "user", "content": query}
            ],
            max_tokens=150
        )
        response_content = chat_completion.choices[0].message.content
    except Exception as e:
        print(f"An error occurred during inference: {e}")
        return
    
    print("AI Response:")
    print(response_content)

In [14]:
default_inference("what is the vacation policy?")


2025-01-28 16:21:31,216 - httpx - INFO - HTTP Request: POST http://localhost:51101/v1/chat/completions "HTTP/1.1 200 OK"


AI Response:
To provide a specific vacation policy, I would need to know which company or organization you are referring to. Generally, vacation policies vary by employer and can include details like the number of days off per year, accrual rates, types of leave, and any approval processes. Could you specify the organization?


In [10]:
rag_inference("what is the vacation policy?", collection_name, client.retrieval)


2025-01-28 16:19:19,824 - httpx - INFO - HTTP Request: POST http://localhost:51101/v1/chat/completions "HTTP/1.1 200 OK"


AI Response:
The vacation policy at FurryFriends Pet Supplies includes the following key points:

1. **Advance Notice:**
   - Requests are encouraged at least two weeks in advance.
   - Extended vacations or significant personal events should be submitted even earlier.

2. **Submission Process:**
   - Use the FurryFriends Employee Portal under the "Time Off" section.
   - Alternatively, submit a written request to your immediate supervisor.

3. **Approval Process:**
   - Supervisors review requests based on staffing needs, seniority, and the order of submission.
   - Confirmation is provided within three business days.
   - Scheduling conflicts are resolved based on order of submission and operational requirements.

4. **Limitations on Consecutive Days Off:**


Referenced Files:
- /Users/tylerhouchin/Desktop/kamiwaza-core/kamiwaza/notebooks/rag-demo/pet_store_docs/time-off-requests.md
- /Users/tylerhouchin/Desktop/kamiwaza-core/kamiwaza/notebooks/rag-demo/pet_store_docs/break-policy.md